In [1]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau



In [2]:
! pip install kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d harshul23/garbage

Saving kaggle.json to kaggle.json
 90% 37.0M/41.0M [00:00<00:00, 112MB/s]
100% 41.0M/41.0M [00:00<00:00, 113MB/s]


In [3]:
!unzip garbage.zip

Archive:  garbage.zip
  inflating: dataset/Dataset/Test/Cardboard/cardboard10.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard100.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard101.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard102.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard103.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard104.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard105.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard106.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard107.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard108.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard109.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard11.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard110.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard111.jpg  
  inflating: dataset/Dataset/Test/Cardboard/cardboard112.jpg  
  inflating: dataset/Dataset/Test/C

In [4]:
!rm /content/dataset/Dataset/Train/cardboard/.ipynb_checkpoints
!rm /content/dataset/Dataset/Train/cardboard/Thumbs.db

rm: cannot remove '/content/dataset/Dataset/Train/cardboard/.ipynb_checkpoints': No such file or directory


In [5]:
from keras.applications.resnet_v2 import ResNet50V2
import tensorflow as tf

model2 = tf.keras.applications.ResNet50V2(input_shape=(128, 128, 3),
    include_top=False,
    weights="imagenet",
    classes=1000,
    classifier_activation="softmax",
)

y=model2.output

y= GlobalAveragePooling2D()(y)
y= BatchNormalization()(y)
y= Dropout(0.5)(y)
y= Dense(1024,activation='relu')(y) 
y= Dense(512,activation='relu')(y) 
y= BatchNormalization()(y)
y= Dropout(0.5)(y)

preds2=Dense(12,activation='softmax')(y) 

94683136/94668760 [==============================] - 1s 0us/step


In [6]:
model_Res=Model(inputs=model2.input,outputs=preds2)
model_Res.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']         

In [7]:
for layer in model_Res.layers[:-8]:
    layer.trainable=False
    
for layer in model_Res.layers[-8:]:
    layer.trainable=True

In [8]:
model_Res.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_Res.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']         

In [9]:
main_dir = "dataset/Dataset"
data2=[]
labels2=[]
random.seed(42)
imagePaths2 = sorted(list(os.listdir(main_dir)))
random.shuffle(imagePaths2)
print(imagePaths2)

['Train', 'Test']


In [10]:
for direc in imagePaths2:
    path=sorted(list(os.listdir("/content/dataset/Dataset/"+direc)))
    for img2 in path:
        path2=sorted(list(os.listdir("/content/dataset/Dataset/"+direc+'/'+img2)))
        for i in path2:
            image2 = cv2.imread(("/content/dataset/Dataset/"+direc+'/'+img2+'/'+i))
            image2 = cv2.resize(image2, (128,128))
            image2 = img_to_array(image2)
            data2.append(image2)
            l2 = label2 = img2
            labels2.append(l2)

In [11]:
data2 = np.array(data2, dtype="float32") / 255.0
labels2 = np.array(labels2)
mlb2 = LabelBinarizer()
labels2 = mlb2.fit_transform(labels2)
print(labels2[0])

[0 0 0 0 0 0 1 0 0 0 0 0]


In [12]:
(xtrain2,xtest2,ytrain2,ytest2)=train_test_split(data2,labels2,test_size=0.4,random_state=42)
print(xtrain2.shape, xtest2.shape)

(1516, 128, 128, 3) (1011, 128, 128, 3)


In [13]:
anne2 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint2 = ModelCheckpoint('model2.h5', verbose=1, save_best_only=True)

datagen2 = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen2.fit(xtrain2)

In [ ]:
history2 = model_Res.fit_generator(datagen2.flow(xtrain2, ytrain2, batch_size=128),
               steps_per_epoch=xtrain2.shape[0] //128,
               epochs=30,
               verbose=2,
               callbacks=[anne2, checkpoint2],
               validation_data=(xtrain2, ytrain2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/30

Epoch 00001: val_loss improved from inf to 1.97882, saving model to model2.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


11/11 - 149s - loss: 2.8603 - accuracy: 0.2817 - val_loss: 1.9788 - val_accuracy: 0.4743 - lr: 0.0010 - 149s/epoch - 14s/step
Epoch 2/30

Epoch 00002: val_loss improved from 1.97882 to 1.24300, saving model to model2.h5
11/11 - 134s - loss: 1.7837 - accuracy: 0.4935 - val_loss: 1.2430 - val_accuracy: 0.5963 - lr: 0.0010 - 134s/epoch - 12s/step
Epoch 3/30


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation losses')
plt.xlabel('epoch')

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation accuracy')
plt.xlabel('epoch')